In [ ]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.utilities import DuckDuckGoSearchAPIWrapper
import wikipedia
import requests
from bs4 import BeautifulSoup

In [2]:
llm = ChatOpenAI(temperature=0.2, model="gpt-3.5-turbo")

# ✅ 2. Wikipedia 검색 툴
def search_wikipedia(query: str) -> str:
    try:
        summary = wikipedia.summary(query, sentences=5)
        return summary
    except Exception as e:
        return f"Wikipedia Error: {e}"
      
# ✅ 3. DuckDuckGo 검색 툴
ddg = DuckDuckGoSearchAPIWrapper()
def search_duckduckgo(query: str) -> str:
    results = ddg.results(query, max_results=5)
    return "\n".join([r["link"] for r in results])      
  
# ✅ 4. 웹사이트 스크래핑 툴
def scrape_website(url: str) -> str:
    try:
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")
        # 주요 내용만 추출
        for script in soup(["script", "style"]):
            script.decompose()
        text = soup.get_text(separator=" ")
        return text[:3000]  # 너무 길지 않게 자름
    except Exception as e:
        return f"Scraping Error: {e}"
      
# ✅ 5. 텍스트 저장 툴
def save_to_txt(content: str) -> str:
    with open("research_output.txt", "w", encoding="utf-8") as f:
        f.write(content)
    return "Saved research to research_output.txt ✅"

# ✅ 6. LangChain용 커스텀 도구 정의
tools = [
    Tool.from_function(
        name="WikipediaSearch",
        func=search_wikipedia,
        description="Search for a topic in Wikipedia."
    ),
    Tool.from_function(
        name="DuckDuckGoSearch",
        func=search_duckduckgo,
        description="Search the web using DuckDuckGo and return links."
    ),
    Tool.from_function(
        name="WebScraper",
        func=scrape_website,
        description="Scrape and extract text from a given URL."
    ),
    Tool.from_function(
        name="SaveToTxtFile",
        func=save_to_txt,
        description="Save a string of research notes into a .txt file"
    ),
]

# ✅ 7. 에이전트 초기화
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

# ✅ 8. 실행
query = "Research about the XZ backdoor"
result = agent.run(query)
print("\n📄 Final Result:\n", result)



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearch` with `XZ backdoor`


In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.The XZ backdoor refers to a malicious backdoor that was introduced to the Linux buil